In [1]:
from data import read_texts
from lm import *
import numpy as np
import math
data1 = read_texts("data/corpora.tar.gz", "brown")
data2=read_texts("data/corpora.tar.gz", "gutenberg")
data3=read_texts("data/corpora.tar.gz", "reuters")

	test: brown/brown.test.txt
	train: brown/brown.train.txt
	dev: brown/brown.dev.txt
brown  read. train: 39802 dev: 8437 test: 8533
	dev: gutenberg/gutenberg.dev.txt
	train: gutenberg/gutenberg.train.txt
	test: gutenberg/gutenberg.test.txt
gutenberg  read. train: 68740 dev: 14729 test: 14826
	dev: reuters/reuters.dev.txt
	test: reuters/reuters.test.txt
	train: reuters/reuters.train.txt
reuters  read. train: 38169 dev: 8082 test: 8214


In [2]:
unigram=Unigram()
unigram.fit_corpus(data1.train)
x=list(unigram.model.keys())
c=0;modelled_unk=[]
for i in x:
    if unigram.model[i]==1 and i[0]=='Z':
        modelled_unk.append(i)
        c+=1

In [15]:
#import pdb; pdb.set_trace()
class Trigram(LangModel):
    def __init__(self, backoff = 0.000001, modelled_unk=[]):
        self.model = dict()
        self.lbackoff = log(backoff, 2)
        self.modelled_unk=modelled_unk
        
    def fit_sentence(self, sentence):
        sentence.append('END_OF_SENTENCE')
        for i in range(len(sentence)):
            if sentence[i] in self.modelled_unk:
                sentence[i]='UNK'
                
        for i in range(len(sentence)):
            if i==0:
                if ('*', '*') in self.model.keys():
                    if sentence[i] in self.model[('*','*')].keys():
                        self.model[('*','*')][sentence[i]]+=1
                    else:
                        self.model[('*','*')][sentence[i]]=1
                else:
                    self.model[('*','*')]={}
                    self.model[('*','*')][sentence[i]]=1
            elif i==1:
                if ('*', sentence[0]) in self.model.keys():
                    if sentence[i] in self.model[('*',sentence[0])].keys():
                        self.model[('*',sentence[0])][sentence[i]]+=1
                    else:
                        self.model[('*',sentence[0])][sentence[i]]=1
                else:
                    self.model[('*',sentence[0])]={}
                    self.model[('*',sentence[0])][sentence[i]]=1
            else:
                if (sentence[i-2], sentence[i-1]) in self.model.keys():
                    if sentence[i] in self.model[(sentence[i-2], sentence[i-1])].keys():
                        self.model[(sentence[i-2], sentence[i-1])][sentence[i]]+=1
                    else:
                        self.model[(sentence[i-2], sentence[i-1])][sentence[i]]=1
                else:
                    self.model[(sentence[i-2], sentence[i-1])]={}
                    self.model[(sentence[i-2], sentence[i-1])][sentence[i]]=1
    
    
    def cond_logprob(self, word_tuple, pick):
        if word_tuple in self.model.keys():
            D=self.model[word_tuple] 
            return D.get(pick,0)/sum(D.values())
        else:
            return self.lbackoff   
    
    def cond_logprob_1(self, word_tuple, previous):
        if word_tuple in self.model.keys():
            return self.sample(self.model[word_tuple])
        else:
            return self.lbackoff
        
    def sample(self, D):
        values=np.array(list(D.values()))
        prob= values/sum(D.values())
        val=np.random.choice(len(D.keys()),1,p=prob)
        return list(D.keys())[int(val)]
    
    def generate_next(self):
        i=('*','*');sent=[]
        while i[1]!='END_OF_SENTENCE':
            sent.append(i[0])
            i=(i[1],self.cond_logprob_1(i, 0))
            s=' '.join(sent)
        return s
    def perplexity(self, corpus):
        """Computes the perplexity of the corpus by the model.

        Assumes the model uses an EOS symbol at the end of each sentence.
        """
        return pow(2.0, self.entropy(corpus))

    def entropy(self, corpus):
        num_words = 0.0
        sum_logprob = 0.0
        for s in corpus:
            num_words += len(s) + 1 # for EOS
            sum_logprob += self.logprob_sentence_tri(s)
        num_words=num_words-len(self.modelled_unk)+1
        return -(1.0/num_words)*(sum_logprob)
    
    def logprob_sentence_tri(self, sentence):
        p=1.0
        sentence=['*','*'] + sentence + ['END_OF_SENTENCE']
        
        for i in range(2, len(sentence)):
            p*=self.cond_logprob((sentence[i-2], sentence[i-1]), sentence[i])
        return math.log(p,2)
    

class Bigram(LangModel):
    def __init__(self, backoff = 0.000001, modelled_unk=[]):
        self.model = dict()
        self.lbackoff = log(backoff, 2)
        self.modelled_unk=modelled_unk
        
    def fit_sentence(self, sentence):
        sentence.append('END_OF_SENTENCE')
        sentence=['*','*']+sentence
        for i in range(len(sentence)):
            if sentence[i] in self.modelled_unk:
                sentence[i]='UNK'
                
        for i in range(1,len(sentence)):
            if sentence[i-1] in self.model.keys():
                if sentence[i] in self.model[sentence[i-1]].keys():
                    self.model[sentence[i-1]][sentence[i]]+=1
                else:
                    self.model[sentence[i-1]][sentence[i]]=1
            else:
                self.model[sentence[i-1]]={}
                self.model[sentence[i-1]][sentence[i]]=1
                
    def cond_logprob_1(self, word_tuple, previous):
        if word_tuple in self.model.keys():
            return self.sample(self.model[word_tuple])
        else:
            return 'UNK'
    
    def cond_logprob(self, word_tuple, pick):
        if word_tuple in self.model.keys():
            D=self.model[word_tuple] 
            return D.get(pick,0)/sum(D.values())
        else:
            return self.lbackoff 
        
    def sample(self, D):
        values=np.array(list(D.values()))
        prob= values/sum(D.values())
        val=np.random.choice(len(D.keys()),1,p=prob)
        return list(D.keys())[int(val)]

class linear_interpolation(LangModel): 
    def __init__(self, gamma, t_model, b_model, u_model, modelled_unk=[]):
        self.tri=t_model
        self.bi=b_model
        self.uni= u_model
        self.gamma= gamma
        self.modelled_unk=modelled_unk
        self.lbackoff=0.0000001
    def calc_lambdas(self, c_u_v, c_v):
        l1=c_u_v/(c_u_v+ self.gamma)
        l2=(1-l1)*c_v/(c_v+ self.gamma)
        l3=1-l1-l2
        return (l1,l2,l3)
    
    def perplexity(self, corpus):
        """Computes the perplexity of the corpus by the model.

        Assumes the model uses an EOS symbol at the end of each sentence.
        """
        return pow(2.0, self.entropy(corpus))

    def entropy(self, corpus):
        num_words = 0.0
        sum_logprob = 0.0
        for s in corpus:
            num_words += len(s) + 1 # for EOS
            sum_logprob += self.logprob_sentence_comb(s)
        num_words=num_words-len(self.modelled_unk)+1
        return -(1.0/num_words)*(sum_logprob)
    
    def logprob_sentence_comb(self, sentence):
        p=0
        sentence=['*','*'] + sentence + ['END_OF_SENTENCE']
        
        for i in range(len(sentence)):
            if (sentence[i] in self.modelled_unk) or (sentence[i] not in self.uni.model.keys()):
                sentence[i]='UNK'
                
        for i in range(2, len(sentence)):
            if (sentence[i-2], sentence[i-1]) in self.tri.model.keys():
                c_u_v= sum(self.tri.model[(sentence[i-2], sentence[i-1])].values())
            else:
                c_u_v=0
            if sentence[i-1] in self.bi.model.keys():
                c_v=sum(self.bi.model[sentence[i-1]].values())
            else:
                c_v=0
            l1,l2,l3=self.calc_lambdas(c_u_v, c_v)
            temp=(l1*self.tri.cond_logprob((sentence[i-2], sentence[i-1]), sentence[i])+l2*self.bi.cond_logprob(sentence[i-1], sentence[i])+ l3*self.uni.model[sentence[i]]/sum(list(self.uni.model.values())))
            p=p+math.log(temp,2)
        return p
    
    def gen_next(self):
        i=('*','*');sent=[]
        while i[1]!='END_OF_SENTENCE':
            sent.append(i[0])
            if i in self.tri.model.keys():
                c_u_v= sum(self.tri.model[i].values())
            else:
                c_u_v=0
            if i[1] in self.bi.model.keys():
                c_v=sum(self.bi.model[i[1]].values())
            else:
                c_v=0
            l1,l2,l3=self.calc_lambdas(c_u_v, c_v)
            a1=self.tri.cond_logprob_1(i, 0)
            a2=self.bi.cond_logprob_1(i[1], 0)
            x=np.random.choice(len(self.uni.model.keys()),1,p=np.array(list(self.uni.model.values()))/sum(self.uni.model.values()))
            a3=list(self.uni.model.keys())[int(x)]
            x= np.random.choice(3,1,p=[l1,l2,l3])
            if x==0:
                next_word=a1
            elif x==1:
                next_word=a2
            else:
                next_word=a3
            i=(i[1],next_word)
            s=' '.join(sent)
        return s
        
trigram=Trigram(modelled_unk=modelled_unk)
unigram=Unigram(modelled_unk=modelled_unk)
bigram=Bigram(modelled_unk=modelled_unk)
trigram.fit_corpus(data1.train+data2.train)
bigram.fit_corpus(data1.train+data2.train)
unigram.fit_corpus(data1.train+data2.train)

In [16]:
L= linear_interpolation(1000, trigram, bigram, unigram, modelled_unk=modelled_unk)

In [17]:
print(L.perplexity(data2.test))

410.377261288678


In [8]:
L.gen_next()

'* * According government after an increase of the plant located in the price on the first quarter when the company will result in five rural Main imposition slaughterhouse Japan would allocate in'

In [10]:
len(data2.train)*0.75


51555.0